In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pyodbc

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_rep_email from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC' and inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
df = df.dropna()
df['inc_rep_email'] = df['inc_rep_email'].str.split("@",n=1,expand=True)[0]
df.head()

In [ ]:
email_counts = df['inc_rep_email'].value_counts(ascending=True)
df_counted = df
for email,count in email_counts.iteritems():
    if count <= 15:
        df_counted.loc[df_counted['inc_rep_email'] == email, 'inc_rep_email'] = 'Other'
email_counts_min = df_counted['inc_rep_email'].value_counts(ascending=True)
email_counts_min = email_counts_min.drop('Other')
print(email_counts_min)

In [ ]:
plt.figure(figsize=(15,15))
plt.barh(email_counts_min.index,email_counts_min.values)

In [ ]:
people_per_counts = email_counts.value_counts()[email_counts.unique()]
print(people_per_counts)
bins = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,30,40,50,80,400]
people_per_counts_binned = people_per_counts.groupby(pd.cut(people_per_counts.index,bins)).sum()
print(people_per_counts_binned)
plt.figure(figsize=(14,14))
string_bins = people_per_counts_binned.index.astype(str)
print(string_bins)
plt.barh(string_bins,people_per_counts_binned.values)

In [ ]:
cumulative_num_people = []
cumulative_percentages = []
num_people = people_per_counts.values.sum()
percentage = 100
for i in range(len(people_per_counts)):
    cumulative_num_people.append(str(num_people))
    num_people -= people_per_counts.values[i]
    cumulative_percentages.append(percentage)
    percentage -= 100*people_per_counts.index[i]*people_per_counts.values[i]/email_counts.values.sum()
red_percentages = [100-percentage for percentage in cumulative_percentages]

In [ ]:
plt.figure(figsize=(14,14))
plt.barh(cumulative_num_people,cumulative_percentages,color='g')
plt.barh(cumulative_num_people,red_percentages,left=cumulative_percentages,color='r')
plt.ylabel('number of people reporting the events')
for i, percentage in enumerate(cumulative_percentages):
    plt.text(percentage,i-0.2,"{0:.1f}% ".format(percentage))
    if int(cumulative_num_people[i])!=1 and i>2: 
        plt.text(69,i-0.2,"the same "+cumulative_num_people[i]+" people reported "+"{0:.1f}% ".format(percentage)+" of events")
plt.text(69,len(cumulative_percentages)-1-0.2,"the same 1 person reported "+"{0:.1f}% ".format(percentage)+" of events");

In [ ]:
plt.rcParams.update({'font.size': 9})
plt.figure(figsize=(20,14))
plt.bar(cumulative_num_people,cumulative_percentages,color='g')
plt.bar(cumulative_num_people,red_percentages,bottom=cumulative_percentages,color='r')
plt.xlabel('number of people reporting the events');
plt.xlim(-0.5,47.5)
plt.gca().invert_xaxis()
for i, percentage in enumerate(cumulative_percentages):
    plt.text(i-0.1,percentage+0.1,"{0:.0f}% ".format(percentage),ha='center')